In [1]:
import os 
import sys
sys.path.append("../utils")
import xarray as xr
import pandas as pd
from tools_utils import CDO, NCKS, NCAP2

In [2]:
ifs_dir = "/p/scratch/deepacf/maelstrom/maelstrom_data/ifs_hres/orig"
radklim_dir = "/p/scratch/deepacf/deeprain/radklim_process/netcdf/remapped/"

outdir_base = "/p/scratch/deepacf/maelstrom/maelstrom_data/ap5_michael/preprocessed_era5_radklim"

sf_vars = ["cp", "lsp", "cape", "tclw", "tcwv", "sp","tisr"]
pl_vars = ["u", "v"]#{"u": {"pl": 70000.}, "v": {"pl": 70000.}}
radklim_vars = "YW_hourly"

years = [2016]

lonlat_box = [8.0, 9.6, 50.001, 51.201]

ll_box_str = "{0:.3f}, {1:.3f}, {2:.3f}, {3:.3f}".format(*lonlat_box)
rd_vars_lower = radklim_vars.lower()

print(ll_box_str)

8.000, 9.600, 50.001, 51.201


In [3]:
# initialize tools
cdo, ncks, ncap2 = CDO(), NCKS(), ncap2()

AttributeError: 'int' object has no attribute 'partition'

In [ ]:
print(xr.__file__)

In [ ]:
for yr in years:
    for m in range(1, 13):
        # create output directory incl. tmp
        outdir = os.path.join(outdir_base, "{0:d}-{1:0d}".format(yr, mm))
        outdir_tmp = os.path.join(outdir, "tmp")
        os.makedirs(outdir, exist_ok=True)
        os.makedirs(outdir_tmp, exist_ok=True)
        
        # get current IFS forecast directory
        dirr_curr_ifs = os.path.join(ifs_data_dir, "{0:d}/{0:d}-{1:0d}".format(yr, mm))
        
        # construct time-arrays for which data is required
        ld = last_day_of_month(dt.datetime.strptime("{0:d}{1:0d}01", "%Y%m%d"))
        init_runs = pd.date_range("{0:d}-{1:0d}-01".format(yr, mm),
                                  "{0:d}-{1:0d}-{3:d} 12:00".format(yr, mm, ld), freq="12H")
        radklim_dates = pd.date_range(init_runs[0] + dt.timedelta(hours=6), init_runs + dt.timedelta(hours=6),
                                      freq="1H")
        # loop over all IFS forecast-files
        for ir in init_runs:
            ifs_sf_file = os.path.join(dirr_curr_ifs, "sfc_{0}.nc".format(ir.strftime("%Y%m%d_%H")))
            sf_file_out = os.path.join(outdir, "sfc_{0}.nc".format(ir.strftime("%Y%m%d%H")))
            
            ifs_pl_file, pl_file_out = ifs_sf_file.replace("sfc", "pl"), sf_file_out("sfc", "pl")
            
            cdo.run([ifs_sf_file, sf_file_out], OrderedDict([("-selname", ",".join(sf_vars)), ("-sellonlatbox", ll_box_str), 
                                                             ("-seltimestep", "6/17")]))
            cdo.run([ifs_pl_file, pl_file_out], OrderedDict([("-selname", ",".join(pl_vars)), ("-sellonlatbox", ll_box_str), 
                                                             ("-seltimestep", "6/17"), ("-sellevel", "70000.")]))
        # merge IFS forecast-files 
        all_sf_files, all_pl_files = glob.glob(os.path.join(outdir, "sfc_*.nc")), glob.glob(os.path.join(outdir, "pl_*.nc"))
        sf_file = os.path.join(outdir, "sf_{0}.nc".format(ir.strftime("%Y-%m")))
        pl_file = sf_file.replace("sf_", "pl_")
                                                        
        cdo.run(all_sf_files + [sf_file], OrderedDict([("mergetime", "")]))
        cdo.run(all_pl_files + [pl_file], OrderedDict([("mergetime", "")]))
        
        # loop over all corresponding RADKLIM timesteps
        for rd in radklim_dates:
            yr_now, mm_now, dd_now, hh_now = rd.strptime("%Y"), rd.strptime("%m"), rd.strptime("%d"), rd.strptime("%H")
            
            curr_file_lres = os.path.join(radlim_dir, "radklim_reg_lres", rd_vars_lower, yr_now,
                                          "{0}-{1}".format(yr_now, mm_now),
                                          "{0}_remapped_radklim_reg_lres_{1}.nc".format(radklim_var, rd.strftime("%Y-%m-%d")))
            file_lres_out = os.path.join(outdir, "radklim_lres_{0}.nc".format(rd.strftime("%Y%m%d%H")))
            
            curr_file_hres = curr_file_lres.replace("radklim_reg_lres", "radklim_reg_hres")
            file_hres_out = file_lres_out.replace("reg_lres", "reg_hres")
            
            rd_dict_cdo = OrderedDict([("-L", ""), ("-seltimestep", rd.strftime("%H")), ("-selname", rd_vars_lower),
                                       ("-sellonlatbox", ll_box_str)])
            cdo.run([curr_file_lres, file_lres_out], rd_dict_cdo)
            cdo.run([curr_file_hres, file_hres_out], rd_dict_cdo)
            
        # merge all RADKLIM files  
        all_rd_lres_files = glob.glob(os.path.join(outdir, "radklim_lres_*.nc"))
        all_rd_hres_files = glob.glob(os.path.join(outdir, "radklim_hres_*.nc"))
        
        rd_lres_file = os.path.join(outdir, "radklim_lres_{0}.nc".format(ir.strftime("%Y-%m")))
        rd_hres_file = rd_lres_file.replace("_lres_", "_hres_")
        
        cdo.run(all_rd_lres_files + [rd_lres_file], OrderedDict([("mergetime", "")]))
        cdo.run(all_rd_hres_files + [rd_hres_file], OrderedDict([("mergetime", "")]))
                                          

        